# 如何合并相同类型的连续消息

:::note
`mergeMessageRuns` 函数可在 `@langchain/core` 版本 `0.2.8` 及以上版本中使用。
:::

某些模型不支持连续传递相同类型的（连续的）消息。

`mergeMessageRuns` 工具可以轻松合并相同类型的连续消息。

## 基本用法

In [1]:
import { HumanMessage, SystemMessage, AIMessage, mergeMessageRuns } from "@langchain/core/messages";

const messages = [
    new SystemMessage("you're a good assistant."),
    new SystemMessage("you always respond with a joke."),
    new HumanMessage({ content: [{"type": "text", "text": "i wonder why it's called langchain"}] }),
    new HumanMessage("and who is harrison chasing anyways"),
    new AIMessage(
        'Well, I guess they thought "WordRope" and "SentenceString" just didn\'t have the same ring to it!'
    ),
    new AIMessage("Why, he's probably chasing after the last cup of coffee in the office!"),
];

const merged = mergeMessageRuns(messages);
console.log(merged.map((x) => JSON.stringify({
    role: x._getType(),
    content: x.content,
}, null, 2)).join("\n\n"));

{
  "role": "system",
  "content": "you're a good assistant.\nyou always respond with a joke."
}

{
  "role": "human",
  "content": [
    {
      "type": "text",
      "text": "i wonder why it's called langchain"
    },
    {
      "type": "text",
      "text": "and who is harrison chasing anyways"
    }
  ]
}

{
  "role": "ai",
  "content": "Well, I guess they thought \"WordRope\" and \"SentenceString\" just didn't have the same ring to it!\nWhy, he's probably chasing after the last cup of coffee in the office!"
}


请注意，如果要合并的消息中的内容是内容块的列表，则合并后的消息将具有内容块的列表。如果要合并的两条消息的内容都是字符串，则它们的内容会用换行符连接起来。

## 链式操作

`mergeMessageRuns` 可以以命令式（如上所示）或声明式的方式使用，便于与链中的其他组件组合使用：

In [2]:
import { ChatAnthropic } from "@langchain/anthropic";
import { mergeMessageRuns } from "@langchain/core/messages";

const llm = new ChatAnthropic({ model: "claude-3-sonnet-20240229", temperature: 0 });
// Notice we don't pass in messages. This creates
// a RunnableLambda that takes messages as input
const merger = mergeMessageRuns();
const chain = merger.pipe(llm);
await chain.invoke(messages);

AIMessage {
  lc_serializable: true,
  lc_kwargs: {
    content: [],
    additional_kwargs: {
      id: 'msg_01LsdS4bjQ3EznH7Tj4xujV1',
      type: 'message',
      role: 'assistant',
      model: 'claude-3-sonnet-20240229',
      stop_reason: 'end_turn',
      stop_sequence: null,
      usage: [Object]
    },
    tool_calls: [],
    usage_metadata: { input_tokens: 84, output_tokens: 3, total_tokens: 87 },
    invalid_tool_calls: [],
    response_metadata: {}
  },
  lc_namespace: [ 'langchain_core', 'messages' ],
  content: [],
  name: undefined,
  additional_kwargs: {
    id: 'msg_01LsdS4bjQ3EznH7Tj4xujV1',
    type: 'message',
    role: 'assistant',
    model: 'claude-3-sonnet-20240229',
    stop_reason: 'end_turn',
    stop_sequence: null,
    usage: { input_tokens: 84, output_tokens: 3 }
  },
  response_metadata: {
    id: 'msg_01LsdS4bjQ3EznH7Tj4xujV1',
    model: 'claude-3-sonnet-20240229',
    stop_reason: 'end_turn',
    stop_sequence: null,
    usage: { input_tokens: 84, outpu

查看 [LangSmith 跟踪](https://smith.langchain.com/public/48d256fb-fd7e-48a0-bdfd-217ab74ad01d/r)，我们可以看到消息在传递给模型之前会被合并。

仅查看合并过程，我们可以看到它是一个Runnable对象，可以像所有Runnable一样被调用：

In [3]:
await merger.invoke(messages)

[
  SystemMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: "you're a good assistant.\nyou always respond with a joke.",
      name: undefined,
      additional_kwargs: {},
      response_metadata: {},
      id: undefined
    },
    lc_namespace: [ 'langchain_core', 'messages' ],
    content: "you're a good assistant.\nyou always respond with a joke.",
    name: undefined,
    additional_kwargs: {},
    response_metadata: {},
    id: undefined
  },
  HumanMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: [Array],
      name: undefined,
      additional_kwargs: {},
      response_metadata: {},
      id: undefined
    },
    lc_namespace: [ 'langchain_core', 'messages' ],
    content: [ [Object], [Object] ],
    name: undefined,
    additional_kwargs: {},
    response_metadata: {},
    id: undefined
  },
  AIMessage {
    lc_serializable: true,
    lc_kwargs: {
      content: `Well, I guess they thought "WordRope" and "SentenceString" just didn't hav

## API 参考

有关所有参数的完整描述，请前往 [API 参考](https://api.js.langchain.com/functions/langchain_core.messages.mergeMessageRuns.html)。